In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score

import xgboost as xgb
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [22]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('final_proteins_dataframe.csv', index_col=0)
df

,pdb_id,chain_id,sequence,residue,residue_number,res_index,pdb_res_index,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,U,avg_hydrophobicity,avg_hydrophilicity,avg_nci,avg_polarity,avg_polarizability,avg_sasa,avg_vol,hydrophobicity,hydrophilicity,NCI,polarity,polarizability,SASA,vol,is_charged,tm_segment
0,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,D,3,0,5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.003333,0.433333,0.040025,9.400000,0.119333,1.595333,54.283333,-0.90,3.0,-0.023820,13.0,0.105,1.587,40.0,1,0
1,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,L,10,1,6,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,-0.502000,1.120000,0.056747,13.380000,0.201400,2.426400,86.140000,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0,0
2,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,L,10,2,7,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,-0.107500,0.200000,0.149690,16.700000,0.304750,3.486000,131.525000,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0,0
3,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,N,12,3,8,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.656667,3.533333,0.161263,31.300000,0.412333,5.062000,166.600000,-0.78,0.2,0.005392,11.6,0.134,1.655,58.7,0,0
4,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,D,3,4,9,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,-3.675000,7.400000,0.326357,52.750000,0.658000,8.283000,251.950000,-0.90,3.0,-0.023820,13.0,0.105,1.587,40.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138182,7b0o,A,LYFQGHMDRLITLVVSYSIAFSIFALATMAVVYGKWLYYFEIDFLN...,V,18,211,206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,-2.920000,2.650000,0.182683,41.500000,1.173500,11.068500,501.550000,1.08,-1.5,0.057004,5.9,0.140,1.645,71.5,0,0
138183,7b0o,A,LYFQGHMDRLITLVVSYSIAFSIFALATMAVVYGKWLYYFEIDFLN...,K,9,212,207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,2.0,-2.300000,2.366667,0.104565,26.033333,0.720333,6.735333,303.200000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1,0
138184,7b0o,A,LYFQGHMDRLITLVVSYSIAFSIFALATMAVVYGKWLYYFEIDFLN...,K,9,213,208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,3.0,-1.790000,2.350000,0.070333,17.975000,0.465750,4.459500,198.075000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1,0
138185,7b0o,A,LYFQGHMDRLITLVVSYSIAFSIFALATMAVVYGKWLYYFEIDFLN...,K,9,214,209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,4.0,-0.926000,1.280000,0.047549,12.280000,0.314400,3.055600,137.460000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1,0


In [23]:
X = df.iloc[:, 4:43]
X = X.drop(['pdb_res_index'], axis=1)
y = df['tm_segment']
X

,residue_number,res_index,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,U,avg_hydrophobicity,avg_hydrophilicity,avg_nci,avg_polarity,avg_polarizability,avg_sasa,avg_vol,hydrophobicity,hydrophilicity,NCI,polarity,polarizability,SASA,vol,is_charged
0,3,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.003333,0.433333,0.040025,9.400000,0.119333,1.595333,54.283333,-0.90,3.0,-0.023820,13.0,0.105,1.587,40.0,1
1,10,1,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,-0.502000,1.120000,0.056747,13.380000,0.201400,2.426400,86.140000,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0
2,10,2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,-0.107500,0.200000,0.149690,16.700000,0.304750,3.486000,131.525000,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0
3,12,3,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.656667,3.533333,0.161263,31.300000,0.412333,5.062000,166.600000,-0.78,0.2,0.005392,11.6,0.134,1.655,58.7,0
4,3,4,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,-3.675000,7.400000,0.326357,52.750000,0.658000,8.283000,251.950000,-0.90,3.0,-0.023820,13.0,0.105,1.587,40.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138182,18,211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,-2.920000,2.650000,0.182683,41.500000,1.173500,11.068500,501.550000,1.08,-1.5,0.057004,5.9,0.140,1.645,71.5,0
138183,9,212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,2.0,-2.300000,2.366667,0.104565,26.033333,0.720333,6.735333,303.200000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1
138184,9,213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,3.0,-1.790000,2.350000,0.070333,17.975000,0.465750,4.459500,198.075000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1
138185,9,214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,4.0,-0.926000,1.280000,0.047549,12.280000,0.314400,3.055600,137.460000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1


In [30]:
# Data normilizations
scaler = StandardScaler()
scaler.fit(X)
scaled = scaler.fit_transform(X)
scaled_df = pd.DataFrame(scaled, columns=X.columns)
scaled_df

,residue_number,res_index,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,U,avg_hydrophobicity,avg_hydrophilicity,avg_nci,avg_polarity,avg_polarizability,avg_sasa,avg_vol,hydrophobicity,hydrophilicity,NCI,polarity,polarizability,SASA,vol,is_charged
0,-1.268103,-1.352340,-0.95271,-0.349348,1.177994,-0.573066,-0.817439,0.216328,-0.410719,-0.869708,-0.562204,0.749250,-0.51974,1.119449,-0.625644,-0.490215,-0.596122,-0.775903,-0.758122,-0.915606,-0.478514,-0.619677,7.745729,-0.654744,1.244695,-0.255210,0.403150,-0.644062,-0.321468,-0.600545,-1.235374,2.047009,-1.111852,2.103075,-0.561158,-0.316691,-0.789417,2.268845
1,-0.037424,-1.345388,-0.95271,-0.349348,2.909585,-0.573066,-0.817439,0.216328,-0.410719,-0.869708,-0.562204,-0.202986,-0.51974,1.119449,-0.625644,-0.490215,0.954054,-0.775903,-0.758122,-0.915606,-0.478514,-0.619677,6.165509,-1.854136,2.168845,0.371952,1.689755,0.511791,0.905097,0.470964,0.827791,-0.777676,0.109580,-1.166404,0.308159,0.440503,0.677918,-0.440753
2,-0.037424,-1.338435,-0.95271,-0.349348,1.177994,-0.573066,-0.817439,0.216328,-0.410719,-0.869708,-0.562204,0.749250,-0.51974,1.119449,0.993896,-0.490215,0.954054,-0.775903,-0.758122,-0.915606,-0.478514,-0.619677,4.585288,-0.917803,0.930663,3.857707,2.763003,1.967404,2.468953,1.997503,0.827791,-0.777676,0.109580,-1.166404,0.308159,0.440503,0.677918,-0.440753
3,0.314199,-1.331482,-0.95271,-0.349348,2.909585,1.022804,-0.817439,0.216328,-0.410719,-0.869708,-0.562204,-0.202986,-0.51974,1.119449,0.993896,-0.490215,0.954054,-0.775903,-0.758122,-0.915606,-0.478514,-0.619677,3.005068,-4.594699,5.416831,4.291752,7.482708,3.482642,4.794960,3.177261,-1.109058,0.399276,-0.639212,1.537980,-0.249921,-0.167013,-0.276535,-0.440753
4,-1.268103,-1.324529,-0.95271,-0.349348,1.177994,1.022804,-0.817439,1.343115,-0.410719,-0.869708,-0.562204,-1.155221,-0.51974,1.119449,0.993896,-0.490215,2.504230,-0.775903,0.607984,-0.915606,-0.478514,-0.619677,1.424848,-9.385146,10.620787,10.483541,14.416796,6.942688,9.548811,6.048035,-1.235374,2.047009,-1.111852,2.103075,-0.561158,-0.316691,-0.789417,2.268845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138182,1.369066,0.114688,-0.95271,-0.349348,-0.553597,-0.573066,-0.817439,-0.910460,-0.410719,-0.869708,4.197291,0.749250,-0.51974,-0.566967,-0.625644,-0.490215,2.504230,-0.775903,-0.758122,-0.915606,-0.478514,2.646265,1.424848,-7.593179,4.227997,5.095106,10.780037,14.203151,13.659911,14.443409,0.848844,-0.601133,0.195850,-0.762764,-0.185527,-0.189024,0.074528,-0.440753
138183,-0.213235,0.121641,-0.95271,-0.349348,-0.553597,-0.573066,-0.817439,-0.910460,-0.410719,-0.869708,2.610793,-0.202986,-0.51974,-0.566967,-0.625644,-0.490215,2.504230,-0.775903,-0.758122,0.237650,-0.478514,2.646265,3.005068,-6.121629,3.846672,2.165324,5.780166,7.820610,7.264621,7.771845,-1.866956,2.047009,-0.439944,1.416888,0.662325,1.160278,0.856192,2.268845
138184,-0.213235,0.128593,-0.95271,-0.349348,-0.553597,-0.573066,-0.817439,-0.910460,-0.410719,-0.869708,2.610793,-0.202986,-0.51974,-0.566967,-0.625644,-0.490215,2.504230,-0.775903,-0.758122,0.237650,-0.478514,1.013294,4.585288,-4.911161,3.824242,0.881505,3.175169,4.234979,3.905735,4.235932,-1.866956,2.047009,-0.439944,1.416888,0.662325,1.160278,0.856192,2.268845
138185,-0.213235,0.135546,-0.95271,-0.349348,-0.553597,-0.573066,-0.817439,-0.910460,-0.410719,-0.869708,2.610793,-0.202986,-0.51974,-0.566967,-0.625644,-0.490215,0.954054,-0.775903,-0.758122,0.237650,-0.478514,1.013294,6.165509,-2.860486,2.384182,0.027000,1.334160,2.103318,1.833729,2.197128,-1.866956,2.047009,-0.439944,1.416888,0.662325,1.160278,0.856192,2.268845


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False) # 7 indicates that always get the same split of data each time this example is executed

In [32]:
# fit model on training data
model = XGBClassifier(objective="binary:logistic", n_estimators = 100, learning_rate = 0.01, max_depth = 12)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# evaluate predictions
accuracy_bl = balanced_accuracy_score(y_test, predictions)
print("Balanced accuracy: %.2f%%" % (accuracy_bl * 100.0))

roc_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print("Roc: %.2f%%" % (roc_score* 100.0))

f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1* 100.0))

Accuracy: 77.18%
Balanced accuracy: 76.95%
Roc: 84.10%
F1 score: 75.07%


In [11]:
inner_metric = 'balanced_accuracy'
outer_metrics = ['balanced_accuracy', 'average_precision', 'f1']

# configure the cross-validation procedure
cv_inner = StratifiedKFold(n_splits=3, shuffle=True)
cv_outer = StratifiedKFold(n_splits=10, shuffle=True)

model = XGBClassifier(objective="binary:logistic")

param_grid = {
  "max_depth": [1, 3, 7, 10],
  "n_estimators": [10, 500, 1000],
  "learning_rate": [0.01, 0.1, 0.2],
}

# define search
rf_grid = GridSearchCV(model, param_grid, scoring='balanced_accuracy', n_jobs=-1, cv=cv_inner, refit=True)

# execute the nested cross-validation
scores = cross_validate(rf_grid, scaled_df, y, scoring='balanced_accuracy', cv=cv_outer, n_jobs=-1)

#Best
rf_grid.fit(X, y)
# params[r] = rf_grid.best_params_
print("Best parameters: ", rf_grid.best_params_)

# scores[r] = rf_grid.best_score_
print("Best score: ", rf_grid.best_score_)

Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532102797542
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532386523935
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532044164002
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.612953192615667
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532610790335
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532122198383
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.612953238518026
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532072701179
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129531919977071
Best paramet

In [ ]:
from sklearn.svm import SVC

svm = SVC(gamma='auto', probability=True)
svm.fit(X_train, y_train)

ypred = svm.predict(X_test)

# evaluate ypred
accuracy = accuracy_score(y_test.values, ypred)
print("Aaccuracy: %.2f%%" % (accuracy * 100.0))

# evaluate ypred
accuracy_bl = balanced_accuracy_score(y_test.values, ypred)
print("Balanced accuracy: %.2f%%" % (accuracy_bl * 100.0))

roc_score = roc_auc_score(y_test.values, svm.predict_proba(X_test)[:, 1])
print("Roc: %.2f%%" % (roc_score* 100.0))

f1 = f1_score(y_test.values, ypred)
print("F1 score: %.2f%%" % (f1* 100.0))

In [27]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
  "learning_rate": [0.01, 0.1, 0.15, 0.2, 0.3],
  "max_depth": [2,4,6,8,10,12,14],
  "num_leaves": [50,60,70,80,90]
}

model = lgb.LGBMClassifier(learning_rate=0.3, max_depth=12, num_leaves=70, num_iterations=500, objective='binary')

# num_round = 10
# lgb.cv(param, train_data, num_round, nfold=5)
model.fit(X_train,y_train, eval_set=[(X_test,y_test),(X_train,y_train)], verbose=20, eval_metric=['auc', 'logloss', 'average_precision'])

[20]	training's auc: 0.88775	training's binary_logloss: 0.424005	training's average_precision: 0.862172	valid_0's auc: 0.843349	valid_0's binary_logloss: 0.490033	valid_0's average_precision: 0.812319
[40]	training's auc: 0.907021	training's binary_logloss: 0.392026	training's average_precision: 0.884741	valid_0's auc: 0.839494	valid_0's binary_logloss: 0.496019	valid_0's average_precision: 0.805588
[60]	training's auc: 0.922899	training's binary_logloss: 0.363493	training's average_precision: 0.908182	valid_0's auc: 0.835846	valid_0's binary_logloss: 0.503054	valid_0's average_precision: 0.801696
[80]	training's auc: 0.936486	training's binary_logloss: 0.337851	training's average_precision: 0.923714	valid_0's auc: 0.831453	valid_0's binary_logloss: 0.511	valid_0's average_precision: 0.79737
[100]	training's auc: 0.94661	training's binary_logloss: 0.316526	training's average_precision: 0.936323	valid_0's auc: 0.829323	valid_0's binary_logloss: 0.51572	valid_0's average_precision: 0.796

LGBMClassifier(learning_rate=0.3, max_depth=12, num_iterations=500,
               num_leaves=70, objective='binary')

In [28]:
print('Training accuracy {:.4f}'.format(model.score(X_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(X_test,y_test)))

Training accuracy 0.9732
Testing accuracy 0.7442
